# 9. 파이썬스러운 객체

파이썬 데이터 모델 >> 사용자 정의 자료형이 내장 자료형처럼 자연스럽게 동작
   ; 덕 타이핑 메커니즘 통해
   
## 9.1. 객체 표현

repr() : 객체를 __개발자__ 가 보고자 하는 형태로 표현한 문자열로 반환
str() : 객체를 __사용자__ 가 보고자 하는 형태로 표현한 문자열로 반환

repr(), str() 메서드 지원하려면 특별 메서드 구현해야

__bytes__() : bytes()에 의해 호출, 객체를 바이트 시퀀스로 표현
__format__() : format(), str.format() 둘 다 __format__() 사용, 특별 포맷 코드 >> 객체 표현 문자열 반환

## 9.2. 벡터 클래스의 부활

예 9-1) v1=Vector2d(3,4)
        v1.x, v1.y : getter 메서드 호출 없이 직접 속성에 접근 가능
        x,y=v1 : 튜플에 언패킹 가능
        v1_clone=eval(repr(v1)) : repr()이 생성자 호출 제대로 표현했는지 확인
        v1==v1_clone : ==을 통해 비교
        print(v1) : str()호출, str()이 순서쌍 생성 
        octects=bytes(v1) : __bytes__() 통해 이진 표현 생성
        abs(v1) : __abs__() 통해 객체 크기 반환
        bool(v1) : __bool__() 통해 객체 크기 0이면 false, 아니면 true 반환

예 9-2) typecode : vector, bytes 변환에 사용하는 클래스 속성
        __init__() : x,y를 float로 변환 >> 부적절한 인수로 객체 생성 방지
        __iter__() : vector2d 반복 가능 >> x,y=my_vector로 언패킹 가능
        __repr__() : {!r}을 각 요소에 repr() 호출해 반환된 문자열로 치환>>문자열 만듦
        반복형 vector2d >> 튜플 >> 순서쌍으로 출력
        typecode >> bytes >> 변환된 bytes와 연결
        튜플 생성 >> 모든 속성 비교
        magnitude() : x,y로 만들어진 직삼각형 사변의 길이
        __bool__() : abs()이용해 사변 길이 계산 >> 불리언형으로 변환
        
## 9.3. 대안 생성자

bytes>>Vector2d
예 9-3) 클래스 메서드 : @classmethod 데커레이터 붙음.
        byte>>typecode 읽기 >> octets으로 memoryview 생성 >> typecode 이용해 형변환
        
## 9.4. @classmethod, @staticmethod

@classmethod : 객체가 아닌 클래스에 연산 수행하는 메서드를 정의
               클래스 자체를 첫번째 인수로 받게 함
               frombytes() 같은 대안 생성자 구현 위해 주로 사용
@staticmethod : 메서드가 특별한 첫번째 인수 받지 않도록 변경
                *정적 메서드 : 모듈 대신 클래스 본체 안에 정의된 평범한 함수*

## 9.5. 포맷된 출력

format() 내장함수, str.format() 메서드 : __format__(format_spec)에 포맷 작업 위입

format(my_obj, format_spec)의 두번째 인수
    format(0.4115226337448559, '0.4f') >>> '0.4115'
str.format()에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 안에서 콜론 뒤의 문자열
    '1 brl= {rate : 0.2f} usd'.format(rate=0.4115226337448559) >> 0.41
    
format( int형, 'b') >> 이진수
format( int , 'x') >> 16진수
format( float , 'f') >> 고정소수점
format( float, '%' ) >> 백분율

__format__() 정의 x >> 상속받은 메서드가 str(my_object) 반환
format(v1, '.3f') >>> type error *v1= Vector2d(3,4)*
 >> vector2d 포맷 간이 언어 구현하면 해결
       벡터 각 요소에 fmt_spec 포맷 적용 >> 포맷된 문자열 '(x,y)'로 만듦
       포맷 명시자 'p' >> 극좌표로 표현하게 angle() 구현
       __format__ 수정
      
## 9.6. 해시 가능한 Vector2d

Vector2d는 해시 불가능 >> 집합 안의 항목으로 사용 불가능
__hash__() 구현해야 Vector2d 해시 가능

예) x,y를 읽기 전용 속성으로 만들기 >> vector2d를 불변형으로
    __hash__() : int 반환해야, 동일한 객체는 동일한 해시값 가져야 >> __eq__() 사용하는 객체 속성 이용 >> 해시 계산
                    XOR(^) 사용
                    
특별 메서드 구현 >> 기능을 완전히 갖춘 객체 만들 수 있음.

## 9.7. 파이썬에서의 비공개 속성과 보호된 속성

private 수정자 없음 >> 비공개 변수 생성 불가능
비공개 속성을 변경하지 못하게 하는 메커니즘 있음. 

이름 장식 : 속성 정의 시 이름 충돌 안 일어나게 속성명을 __mood 나 __mood_ 형식으로 정의 >>언더바, 크래스명 변수명 앞에 붙여 __dict__에 저장
            ex) Dog 클래스 상속하는 Beagle 클래스 >> _Dog__mood, _Beagle__mood
            
속성명 앞에 언더바 하나 : 보호된 속성/ 비공개 속성이라 부름,  보편적으로 보호하게 개발함

Vector2d 요소 : 비공개 속성
Vector2d 객체 : 불변형

## 9.8. __slots__ 클래스 속성으로 공간 절약

파이썬 : 객체 속성을 __dict__ 라는 딕셔너리형 속성에 저장
딕셔너리 : 내부에 해시 테이블 유지 >> 빠른 접근 속도, 메모리 사용 큼

__slots__ 속성 : 객체 속성을 튜플에 저장하게 함.
            정의 : 이 이름의 클래스 속성 생성>> 객체 속성 식별자 담은 문자열의 반복형 할당
                   불변형인 튜플 사용 >> __slots__ 정의 변경할 수 없음
                   
1천만개의 vector2d 객체 
   __dict__ 사용 : 스크립트 램 사용량 1.5GB
   __slots__ 사용 :  655MB

__dict__ 문자열을 __slots__ 리스트에 추가하면 __slots__ 지정한 속성은 튜플에 저장. but, 동적으로 생성한 속성은 __dict__에 저장 >>> __slots__ 사용 의미 없어짐.

객체가 약한 참조 지원 >> __weakref__ 속성 필요
       클래스가 __slots__ 정의 , 이 클래스 객체를 약한 참조 대상 되게 하려면 __weakref__를 __slots__ 리스트에 추가해야.
       
### 9.8.1. __slots__ 사용시 주의점

인터프리터는 상속된 __slots__ 속성 무시하므로 각 클래스마다 __slots__ 속성 다시 정의해야 함.
__dict__를 __slots__ 에 추가하지 않는 객체는 __slots__ 에 나열된 속성만 가질 수 있음.
__weakref__ 를 __slots__ 에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.

## 9.9. 클래스 속성 오버라이드

파이썬은 클래스 속성을 객체 속성의 기본값으로 사용함.

객체 속성 생성해 오버라이드
크래스 상속해 서브 클래스의 클래스 수준에서 덮어써 오버라이드

# 10. 시퀀스 해킹, 해시, 슬라이스

## 10.1. vector : 사용자 정의 시퀀스형

상속 아닌 구성 이용해 벡터 구현
  : 요소들을 실수형 배열에 저장, 벡터 뷸변 균일 시퀀스처럼 작동하게 메서드 구현

## 10.2. vector : vector2d 호환

시퀀스 생성자는 내장 시퀀스처럼 반복형을 인수로 받게 만드는 것이 좋음.

예) 보호된 객체 속성인 self._ components는 벡터 요소를 배열로 저장
    반복할 수 있게 self._ components에 대한 반복자 반환
    reprlib.repr() >> self._ components를 제한된 길이로 표현
    문자열 vector 생성자에 전달 위해 문자열 'array' 제거
    self._ components에서 바로 bytes 객체 생성
    각 요소의 제곱근 구하기 위해 sqrt() 호출
    memoryview를 바로 생성자에게 전달
    
reprliv.repr() : 생략 기호 >> 생성할 문자열 길이 제한 >> 대형 구조체, 재귀적 구조체 안전하게 표현

__str__() , __eq__(), __bool__() 메서드는 vector2d 에서 바뀌지 않음.
    : vector2d 클래스를 반복형으로 만들었기 덕분

vector 클래스가 vector2d 클래스 상속받지 않게 만든 이유
    1. 생성자가 호환되지 않기 때문
    2. vector 클래스가 시퀀스 프로토콜을 구현하는 독자적인 예제가 되기 위해

## 10.3. 프로토콜과 덕 타이핑

파이썬 : 완전히 작동하는 시퀀스형 만들기 위해 어떤 특별한 클래스 상속할 필요 없음. 시퀀스 프로토콜에 따르는 메서드 구현하기만 하면 됨.

프로토콜 : 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스

시퀀스 프로토콜 구현 >> 여러 기능 활용 가능.
클래스가 시퀀스처럼 동작 >> 시퀀스 임.
이를 __덕타이핑__ 이라고 함.

## 10.4. vector : 슬라이스 가능한 시퀀스

vector 슬라이싱 >> 배열
frenchdeck 슬라이싱 >> 리스트

### 10.4.1. 슬라이싱의 작동 방식

dir(slice) 호출 >> indices

S.indices(len) -> (start, stop,stride)
길이가 len 인 시퀀스 S가 나타내는 확장된 슬라이스의 start, stop 인덱스 및 stride 길이를 계산. 경계 벗어난 인덱스는 일반적인 슬라이스 처리방법과 동일하게 잘라냄.
>> 주어진 길이의 시퀀스 경계안에 들어간 start, stop, stride

### 10.4.2. 슬라이스 인식하는 __getitem__()

__getitem__()의 else 문에 'ABC'[1,2]  >> type error : 인덱스는 정수형이어야
    벡터[] 
        [정수] : 단 한 요소의 값을 실수형으로 반환
        [슬라이스] : vector를 새로 만듦
        [길이 1인 슬라이스] : vector 객체 생성
        [인덱스나 슬라이스로 구성된 튜플] : error ; vector는 다차원 인덱싱 지원하지 않기 때문
        
## 10.5 vector : 동적 속성 접근

속성 못 찾음 >> 인터프리터 __getattr__() 호출

__getattr__() : 찾고 있는 속성이 x,y,z,t 문자 중 하나인지 검사, 이 중 하나면 해당 벡터 요소 반환

해당 이름의 속성 찾을 수 있으면 __getattr__() 호출 x
  v.x=10 >> v 객체에 x 속성 추가 >> __getattr__() 호출 x
  
내장된 complex 형의 동작 조사 : complex가 불변형이며 real, imag 두 개의 데이터 속성 갖기 때문

읽기 전용 속성 x,y,z,t 과 혼동 피하기 위해 단일 소문자로 되어 있는 속성의 설정만 막음.

객체 동작의 불일치 피하려면 __getattr__()구현시 __setattr__()도 함께 구현해야

## 10.6. vector : 해싱 및 더 빠른 ==

__eq__() 메서드 >> __hash__() 구현 >> vector 객체 해시 가능

reduce() : 모든 벡터 요소의 해시를 계산하는 연산 수행시 적절
           일련의 값을 하나의 값으로 줄임.
           받는 첫번 째 인수는 두개의 인수를 받는 함수이고, 두번째 인수는 반복형
           
XOR 누적 계산
    1. for 루프, 변수
    2. 익명 함수 이용해 functools.reduce() 호출
    3. 사용자 정의 람다를 operator.xor로 대체해 functools.reduce() 호출
    
맵-리듀스 : 각 항목에 함수를 적용해 새로운 시퀀스 생성(맵) >> 누적 연산 적용(리듀스)    
            맵 : 각 요소에 대한 해시 계산
            리듀스 : 모든 해시에 xor 연산자 적용

__eq__()는 튜플로 비교.>> Vector([1,2]), (1,2) 같다고 판단
다차원 벡터 비교시 비효율적. >> 새롭게 구현
                          len()로 길이 비교 : 길이 다르면 false
                          for 루프 안에 zip() 사용 : 하나라도 다르면 바로 flase
                                >> all(), zip() 로 구현 가능.
*zip()는 가장 짧은 피연산자에서 멈춤 >> 먼저 길이 검사해야*

## 10.7. vector : 포매팅

vector2d의 __format__() : 극좌표, 포맷코드 'p'
vector의 __format__() : (초)구면좌표, 포맷코드 'h'

4차원 벡터 'h' : <r, 각1, 각2, 각3 > 출력
                 r : 크기(abs(v)),    나머지 : 각좌표
                
angle(n) : 특정 좌표의 각좌표 계산하게
angles(): 모든 각좌표의 반복형 반환














